In [1]:
import sys
sys.path.append("..")

In [2]:
from enum import Enum

from resources.utils import get_puzzle_input
from resources.tree import Node

### Part 1

The navigation system's license file consists of a list of numbers (your puzzle input). The numbers define a data structure which, when processed, produces some kind of tree that can be used to calculate the license number.

The tree is made up of nodes; a single, outermost node forms the tree's root, and it contains all other nodes in the tree (or contains nodes that contain nodes, and so on).

Specifically, a node consists of:

```
A header, which is always exactly two numbers:
The quantity of child nodes.
The quantity of metadata entries.
Zero or more child nodes (as specified in the header).
One or more metadata entries (as specified in the header).
Each child node is itself a node that has its own header, child nodes, and metadata. For example:
```

```
2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2
A----------------------------------
    B----------- C-----------
                     D-----
```

In this example, each node of the tree is also marked with an underline starting with a letter for easier identification. In it, there are four nodes:

```
A, which has 2 child nodes (B, C) and 3 metadata entries (1, 1, 2).
B, which has 0 child nodes and 3 metadata entries (10, 11, 12).
C, which has 1 child node (D) and 1 metadata entry (2).
D, which has 0 child nodes and 1 metadata entry (99).
The first check done on the license file is to simply add up all of the metadata entries. In this example, that sum is 1+1+2+10+11+12+2+99=138.
```


In [3]:
test_input = [
    "2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2",
]

In [4]:
def parse_input(input):
    output = []
    for line in input:
        output.extend([int(i) for i in line.split(' ')])
    return output

In [5]:
parse_input(test_input)

[2, 3, 0, 3, 10, 11, 12, 1, 1, 0, 1, 99, 2, 1, 1, 2]

In [6]:
class LicenceNode(Node):
    def __init__(self, name, num_children, num_data):
        self.data = []
        self.num_children = num_children
        self.num_data = num_data
        
        super().__init__(name)
    
    @property
    def consuming(self):
        return self.num_data > len(self.data)
    
    @property
    def expecting(self):
        return self.num_children > len(self.children)
    
    @property
    def value(self):
        """Part 2"""
        if not self.children:
            return sum(self.data)
        
        total = 0
        for datum in self.data:
            if datum > 0 and datum <= len(self.children):
                total += self.children[datum - 1].value
        
        return total

    def add_datum(self, datum):
        self.data.append(datum)

In [7]:
class State(Enum):
    NUM_CHILDREN = 1
    NUM_DATA = 2
    DATA = 3

In [8]:
def create_tree(input):
    process_stack = []
    state = State.NUM_CHILDREN
    num_children = 0
    node_num = 0
    root = None
    
    for i in input:
        if state == State.NUM_CHILDREN:
            # New node
            num_children = i
            state = State.NUM_DATA

        elif state == State.NUM_DATA:
            node = LicenceNode(node_num, num_children, i)
            node_num += 1

            if process_stack:
                node.add_parent(process_stack[-1])
            else:
                root = node
            
            process_stack.append(node)
            
            if node.num_children:
                state = State.NUM_CHILDREN
                continue

            state = State.DATA
        else:
            assert state == State.DATA

            while process_stack:
                current_node = process_stack[-1]

                if current_node.consuming:
                    current_node.add_datum(i)
                    break

                if current_node.parent and current_node.parent.expecting:
                    process_stack.pop()
                    num_children = i
                    state = State.NUM_DATA
                    break
                
                process_stack.pop()
                    
    return root

In [9]:
parsed_input = parse_input(test_input)
test_tree = create_tree(parsed_input)

In [10]:
def checksum(root):
    cs = sum(root.data)
    for d in root.decendants:
        cs += sum(d.data)
    
    return cs

In [11]:
checksum(test_tree)

138

In [12]:
puzzle_input = get_puzzle_input('/tmp/day_8.txt')
parsed_puzzle_input = parse_input(puzzle_input)
len(parsed_puzzle_input)

17176

In [13]:
puzzle_tree = create_tree(parsed_puzzle_input)

In [14]:
checksum(puzzle_tree)

43996

### Part 2

The second check is slightly more complicated: you need to find the value of the root node (A in the example above).

The value of a node depends on whether it has child nodes.

If a node has no child nodes, its value is the sum of its metadata entries. So, the value of node B is 10+11+12=33, and the value of node D is 99.

However, if a node does have child nodes, the metadata entries become indexes which refer to those child nodes. A metadata entry of 1 refers to the first child node, 2 to the second, 3 to the third, and so on. The value of this node is the sum of the values of the child nodes referenced by the metadata entries. If a referenced child node does not exist, that reference is skipped. A child node can be referenced multiple time and counts each time it is referenced. A metadata entry of 0 does not refer to any child node.

For example, again using the above nodes:

```
Node C has one metadata entry, 2. Because node C has only one child node, 2 references a child node which does not exist, and so the value of node C is 0.
Node A has three metadata entries: 1, 1, and 2. The 1 references node A's first child node, B, and the 2 references node A's second child node, C. Because node B has a value of 33 and node C has a value of 0, the value of node A is 33+33+0=66.
So, in this example, the value of the root node is 66.
```

What is the value of the root node?

In [15]:
test_tree.value

66

In [16]:
puzzle_tree.value

35189

In [18]:
import time
int(time.time())

1544340993